Recurrent neural networks are used when a data
pattern changes over time. RNNs can be assumed as 
unrolled over time.

RNNs have feedback loops in which the output from the previous firing or time index T is fed as one of the inputs at time index T+1. There might be cases in which the output of the neuron is fed to itself as input. These are well suited for applications involving sequences.

Encoding RNN- Enables network to take an input of the sequence form.

Generating RNN- network basically outputs a sequence of numbers or values, like words in a sentence.

General recurrent neural networks - combination of the preceding two types of RNNs. It's used to generate sequences and, thus, are widely used in NLG (natural language generation) tasks.

# encoder - decoder networks
This uses one network to create an internal representation of the input, or to 'encode' it, and that representation is used as an input for another network to produce the output.

## recursive neural networks
In a recursive neural network, a fixed set of weights is recursively applied onto the network structure and is primarily used to discover, the hierarchy or structure of the data. 

In [7]:
import numpy as np
name = [1,2,3,4,5]
name1 = [1,2.2,4,1,1]
[np.round(i)==j for i,j in zip(name1, name)]

[True, True, False, False, False]

# word vector representations

#### Introduction to word Embedding
#### Neural Language Model

The feedforward neural net language model (FNNLM) proposed by Bengio introduces a feedforward neural network consisting of a single hidden layer than prdicts the future words. 
1. Embedding layer: This keeps a record of the representation of all the words in the training dataset. It is initialized with a set of random weights. The embedding layer is made up of three parts, which includes the size of the vocabulary, the output size of the vector in which all the words will be embedded, and the length of the input sequences to the model.

# word2vec code

In [1]:
import random, collections, math, os, zipfile, time, re
import numpy as np
import tensorflow as tf

In [6]:
from matplotlib import pylab
%matplotlib inline

from six.moves import range
from six.moves.urllib.request import urlretrieve

"""make sure the dataset link is copied correctly"""

dataset_link = 'http://mattmahoney.net/dc/'
zip_file = 'text8.zip'

In [7]:
def data_download(zip_file):
    """downloading the required file"""
    if not os.path.exists(zip_file):
        zip_file, _ = urlretrieve(dataset_link+zip_file, zip_file)
        print("file downloaded successfully")
        
    return None
data_download(zip_file)

file downloaded successfully


In [8]:
"""Extracting the dataset in separate folder"""
extracted_folder = 'dataset'
if not os.path.isdir(extracted_folder):
    with zipfile.ZipFile(zip_file) as zf:
        zf.extractall(extracted_folder)
with open('dataset/text8') as ft_:
    full_text = ft_.read()

In [9]:
def text_processing(ft8_text):
    """Replacing punctuation marks with tokens"""
    ft8_text = ft8_text.lower()
    ft8_text = ft8_text.replace('.', ' <period> ')
    ft8_text = ft8_text.replace(',', ' <comma> ')
    ft8_text = ft8_text.replace('"', ' <quotation> ')
    ft8_text = ft8_text.replace(';', ' <semicolon> ')
    ft8_text = ft8_text.replace('!', ' <exclamation> ')
    ft8_text = ft8_text.replace('?', ' <question> ')
    ft8_text = ft8_text.replace('(', ' <paren_l> ')
    ft8_text = ft8_text.replace(')', ' <paren_r> ')
    ft8_text = ft8_text.replace('--', ' <hyphen> ')
    ft8_text = ft8_text.replace(':', ' <colon> ')
    ft8_text_tokens = ft8_text.split()
    return ft8_text_tokens

ft_tokens = text_processing(full_text)

In [10]:
"""Shortlisting words with frequency more than 7"""
word_cnt = collections.Counter(ft_tokens)
shortlisted_words = [w for w in ft_tokens if word_cnt[w] > 7 ]

In [11]:
print(shortlisted_words[:15])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']


# CBow Model

The continuous bag-of-words mode l shares an architectural similarity to the FNNLM.